In [1]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
try:
    the_collection.drop()
except NameError:
    pass
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = "./jupyter/bin/python"
spark = SparkSession.builder.config("spark.archives", "pyspark_venv.tar.gz#environment").getOrCreate()
import pygeohash as pgh
from pprint import pprint
import ast
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    client = MongoClient() # so we connect on the default host and port
    db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
    the_collection = db.climate  # create a new collection called 'the_collection'
    climate_value = 0
    n_climate = []
    same_location = []
    for record in iter:  
        first_record = ast.literal_eval(record[1])
        if first_record[-1] == 'Producer1':
            climate_value = first_record
            
        else:
            n_climate.append(first_record)
    for record in n_climate:
        if (pgh.encode(float(record[0]), float(record[1]), precision= 3) 
        == pgh.encode(float(climate_value[0]), float(climate_value[1]), precision =3)):
            same_location.append(record)
            
    geo_dictionary = {}         
    for record in same_location:
        encoding_value = pgh.encode(float(record[0]), float(record[1]), precision= 5)  
        if encoding_value in geo_dictionary:
            geo_dictionary[encoding_value].append(record)
        else:
            geo_dictionary[encoding_value] = [record]
    
    for key in geo_dictionary:
        if len(geo_dictionary[key]) == 1:
            fire_value = geo_dictionary[key]
            if float(climate_value[2]) > 20 and float(climate_value[7]) > 180:
            
                # make a record out of the climate and single fire data value
                new_record = { "latitude" : float(fire_value[0]),
                 "longitude": float(fire_value[1]),
                 "datetime" : None,
                 "confidence" : float(fire_value[2]),
                 "surface_temperature_celsius" : float(hotspot_line[3]),
                "date": fire_value[4],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'natural'}
                the_collection.insert(new_record)
                
            else:
               # make a record out of the climate and single fire data value
                new_record = { "latitude" : float(fire_value[0]),
                 "longitude": float(fire_value[1]),
                 "datetime" : None,
                 "confidence" : float(fire_value[2]),
                 "surface_temperature_celsius" : float(hotspot_line[3]),
                "date": fire_value[4],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'other'}
                the_collection.insert(new_record)                
                
        else:
            fire_numbs = geo_dictionary[key]  # this is a list of records
            average_confidence = 0
            average_sur_temp = 0
            # make a record out of the climate data and the average of everything else.
            for record in firenumbs:
                average_confidence += float(record[2])
                average_sur_temp += float(record[3])
            average_confidence = average_confidence / len(fire_numbs)
            average_sur_temp = average_sur_temp / len(fire_numbs)
            
            if float(climate_value[2]) > 20 and float(climate_value[7]) > 180:
                new_record = { "latitude" : float(climate_value[0]),
                 "longitude": float(climate_value[1]),
                 "datetime" : None,
                 "confidence" : average_confidence,
                 "surface_temperature_celsius" : average_sur_temp,
                "date": climate_value[8],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'natural'}
                the_collection.insert(new_record)
                
            else:
                new_record = { "latitude" : float(climate_value[0]),
                 "longitude": float(climate_value[1]),
                 "datetime" : None,
                 "confidence" : average_confidence,
                 "surface_temperature_celsius" : average_sur_temp,
                "date": climate_value[8],
                "air_temperature_celsius" : float(climate_value[2]),
                "relative_humidity": float(climate_value[3]),
                "windspeed_knots" : float(climate_value[4]),
                "max_wind_speed": float(climate_value[5]),
                "precipitation_level": get_prec_number(climate_value[6]),
                "precipitation_type": get_prec_letter(climate_value[6]),
                "GHI_w/m2": float(climate_value[7]),
                "cause": 'other'}
                the_collection.insert(new_record)
            
            

            
    if geo_dictionary == {}:
        # insert the climate data straight away but with no fire records
        no_fire = {
            "latitude" : float(climate_value[0]),
            "longitude": float(climate_value[1]),
             "datetime": None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "date": climate_value[8],
            "air_temperature_celsius" : float(climate_value[2]),
            "relative_humidity": float(climate_value[3]),
            "windspeed_knots" : float(climate_value[4]),
            "max_wind_speed": float(climate_value[5]),
            "precipitation_level": get_prec_number(climate_value[6]),
            "precipitation_type": get_prec_letter(climate_value[6]),
            "GHI_w/m2": float(climate_value[7]),
             "cause": None
        }
        the_collection.insert(new_record)  
    client.close()

n_secs = 10
topic = "same_topic"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary
#kafkaStream.pprint()
lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

KeyboardInterrupt: 

In [ ]:
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
for k in the_collection.find({}):
    pprint(k)